# Applying Classification Modeling
The goal of this week's assessment is to find the model which best predicts whether or not a person will default on their bank loan. In doing so, we want to utilize all of the different tools we have learned over the course: data cleaning, EDA, feature engineering/transformation, feature selection, hyperparameter tuning, and model evaluation. 


#### Data Set Information:

This research aimed at the case of customers default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods. From the perspective of risk management, the result of predictive accuracy of the estimated probability of default will be more valuable than the binary result of classification - credible or not credible clients. Because the real probability of default is unknown, this study presented the novel Sorting Smoothing Method to estimate the real probability of default. With the real probability of default as the response variable (Y), and the predictive probability of default as the independent variable (X), the simple linear regression result (Y = A + BX) shows that the forecasting model produced by artificial neural network has the highest coefficient of determination; its regression intercept (A) is close to zero, and regression coefficient (B) to one. Therefore, among the six data mining techniques, artificial neural network is the only one that can accurately estimate the real probability of default. 

- NT is the abbreviation for New Taiwain. 


#### Attribute Information:

This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. This study reviewed the literature and used the following 23 variables as explanatory variables: 
- X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit. 
- X2: Gender (1 = male; 2 = female). 
- X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others). 
- X4: Marital status (1 = married; 2 = single; 3 = others). 
- X5: Age (year). 
- X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: 
    - X6 = the repayment status in September, 2005; 
    - X7 = the repayment status in August, 2005; . . .;
    - etc...
    - X11 = the repayment status in April, 2005. 
    - The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
- X12-X17: Amount of bill statement (NT dollar). 
    - X12 = amount of bill statement in September, 2005;
    - etc...
    - X13 = amount of bill statement in August, 2005; . . .; 
    - X17 = amount of bill statement in April, 2005. 
- X18-X23: Amount of previous payment (NT dollar). 
    - X18 = amount paid in September, 2005; 
    - X19 = amount paid in August, 2005; . . .;
    - etc...
    - X23 = amount paid in April, 2005. 




You will fit three different models (KNN, Logistic Regression, and Decision Tree Classifier) to predict credit card defaults and use gridsearch to find the best hyperparameters for those models. Then you will compare the performance of those three models on a test set to find the best one.  


## Process/Expectations

- You will be working in pairs for this assessment

### Please have ONE notebook and be prepared to explain how you worked in your pair.

1. Clean up your data set so that you can perform an EDA. 
    - This includes handling null values, categorical variables, removing unimportant columns, and removing outliers.
2. Perform EDA to identify opportunities to create new features.
    - [Great Example of EDA for classification](https://www.kaggle.com/stephaniestallworth/titanic-eda-classification-end-to-end) 
    - [Using Pairplots with Classification](https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166)
3. Engineer new features. 
    - Create polynomial and/or interaction features. 
    - Additionaly, you must also create **at least 2 new features** that are not interactions or polynomial transformations. 
        - *For example, you can create a new dummy variable that based on the value of a continuous variable (billamount6 >2000) or take the average of some past amounts.*
4. Perform some feature selection. 
    
5. You must fit **three** models to your data and tune **at least 1 hyperparameter** per model. 
6. Using the F-1 Score, evaluate how well your models perform and identify your best model.
7. Using information from your EDA process and your model(s) output provide insight as to which borrowers are more likely to deafult


In [1]:
import pandas as pd


## 1. Data Cleaning

In [3]:
df = pd.read_csv('datasets/training_data.csv', index_col=0)
df.head()

X1 X2 X3 X4  X5  X6  X7  X8  X9 X10  ...     X15     X16     X17  \
28835  220000  2  1  2  36   0   0   0   0   0  ...  221193  181859  184605   
25329  200000  2  3  2  29  -1  -1  -1  -1  -1  ...     326     326     326   
18894  180000  2  1  2  27  -2  -2  -2  -2  -2  ...       0       0       0   
690     80000  1  2  2  32   0   0   0   0   0  ...   43882   42256   42527   
6239    10000  1  2  2  27   0   0   0   0   0  ...    5444    2639    2697   

         X18   X19    X20   X21    X22     X23  Y  
28835  10000  8018  10121  6006  10987  143779  1  
25329    326   326    326   326    326     326  0  
18894      0     0      0     0      0       0  0  
690     1853  1700   1522  1548   1488    1500  0  
6239    2000  1100    600   300    300    1000  1  

[5 rows x 24 columns]

Rename columns using data dictionary

In [6]:
df.Y.value_counts() # 1 = default, 0 = did not default

0                             17471
1                              5028
default payment next month        1
Name: Y, dtype: int64

In [26]:
# Split data to be used in the models
X = df.drop('Y', axis = 1) # grabs everything else but target

# Create target variable
y = df['Y'] # y is the column we're trying to predict

In [12]:
rename_dict= {'X1': 'credit_given', 'X2': 'gender', 'X3': 'education','X4': 'married', 
              'X5': 'age', 'X6': 'repayment_sept', 'X7': 'repayment_aug', 
              'X8':'repayment_jul', 'X9': 'repayment_jun', 'X10': 'repayment_may', 
              'X11':'repayment_apr', 'X12': 'bill_sept', 'X13': 'bill_aug', 'X14':'bill_jul',
              'X15': 'bill_jun', 'X16':'bill_may', 'X17':'bill_apr', 'X18': 'past_payment_sept',
              'X19': 'past_payment_aug', 'X20':'past_payment_jul', 'X21': 'past_payment_jun',
              'X22': 'past_payment_may', 'X23': 'past_payment_apr'}
X=X.rename(columns=rename_dict)

In [32]:
df.astype(float)

ValueError: could not convert string to float: 'LIMIT_BAL'

## 2. EDA

## 3. Feature Engineering

## 4. Feature Selection

## 5. Model Fitting and Hyperparameter Tuning
KNN, Logistic Regression, Decision Tree

## 6. Model Evaluation

## 7. Final Model